In [2]:
# Get the interactive Tools for Matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

import plotly.graph_objects as go
import plotly.express as px



# Get tools to download files and load them 
import pickle
import urllib.request
from os.path import exists as check_path
from os import makedirs

# Get tools to performe analysis
import numpy as np
from heapq import heappushpop
from sklearn.decomposition import PCA

In [3]:
def download_files_from_github(file_target_dir):
    main_url = 'https://raw.githubusercontent.com/Goussha/word-vector-visualization/master/'
    if not check_path(file_target_dir):
        makedirs(file_target_dir)
    
    urls = [main_url+'file{}.p'.format(x) for x in range(1,9)]
    file_names = [file_target_dir+'file{}.p'.format(x) for x in range(1,9)]
    for file_name, url in zip(file_names, urls):
        if not check_path(file_name):
            print ("Downloading file: ",file_name)
            filename, headers = urllib.request.urlretrieve(url, filename=file_name)
        else:
            print('Allready exists: {}'.format(file_name))

In [4]:
def load_word2vecfiles(file_target_dir):
    word_dict_loded = {}
    for file_num in range(1,9):
        full_file_name = file_target_dir+'file{}.p'.format(file_num)
        print('Loading file: {}'.format(full_file_name))
        with open(full_file_name, 'rb') as fp:
            data = pickle.load(fp)
        word_dict_loded.update(data)
    return word_dict_loded

In [5]:
file_target_dir = "./tmp/"

#Download files
download_files_from_github(file_target_dir)
#Load files and create dict
word_dict = load_word2vecfiles(file_target_dir)

Loading file: ./tmp/file1.p
Loading file: ./tmp/file2.p
Loading file: ./tmp/file3.p
Loading file: ./tmp/file4.p
Loading file: ./tmp/file5.p
Loading file: ./tmp/file6.p
Loading file: ./tmp/file7.p
Loading file: ./tmp/file8.p


## cosine similarity
- reflects the degree of similarity between two vectors

In [6]:
def cosine_similarity(u, v):
    distance = 0.0
    epsilon = 1e-10 # prevent deviding by 0
    dot = np.dot(u.T, v)
    # Compute the L2 norm of u & v
    norm_u = np.sqrt(np.sum(u**2))
    norm_v = np.sqrt(np.sum(v**2))
    cosine_similarity = dot/((norm_u*norm_v)+epsilon)
    return cosine_similarity    

## most k similar
- find the most similar word to the input word by calculating the cosine similarity between the word vector and the other word vectors and returning K most similar words

In [10]:
def most_k_similar(word_in, word_dict,k=1):
    words = word_dict.keys()
    word_vec = word_dict[word_in]
    most_similars_heap = [(-100, '') for _ in range(k)]
    for w in words:
        if w==word_in:
            continue
        cosine_sim = cosine_similarity(word_vec, word_dict[w])
        heappushpop(most_similars_heap, (cosine_sim, w))
    most_similars_tuples = [tup for tup in most_similars_heap]
    _, best_words = zip(*most_similars_tuples)
    return best_words

## doesn't match
- takes a list of words and returns the word the doesnt match by comparing cousine similarities each word and all the other words and returning the words with the lowest score

In [11]:
def doesnt_match(words, word_dict):
    dots_tot = []
    for w in words:
        dots = 0
        for w2 in words:
            if w2==w:
                continue
            v = word_dict[w]
            u = word_dict[w2]
            dots = dots + cosine_similarity(v, u)
        dots.tot.append(dots)
    return (words[np.argmin(dots_tot)])